In [ ]:
print("\n READ *readme.txt* file before running this program")
print("\n-------- If any issues contact @ areddy.ragini@gmail.com----------\n")
import pandas as pd
import io
import re
import ipywidgets as widgets
from IPython.display import display
from google.colab import files

def extract_numbers_with_prefix(value):
    """Extract numbers while preserving prefixes."""
    if isinstance(value, (int, float)):
        return str(int(value)) if not pd.isna(value) else None
    elif isinstance(value, str):
        match = re.match(r'(\D*)(\d+)', value)
        if match:
            prefix, number = match.groups()
            return f"{prefix}{int(number)}"  # Maintain original prefix
    return None

def process_file(uploaded_file):
    """Process the uploaded Excel file, always reading Sheet1."""
    try:
        uploaded_filename = list(uploaded_file.keys())[0]  # Get the filename
        file_content = uploaded_file[uploaded_filename]['content']
        file_stream = io.BytesIO(file_content)

        # Read Sheet1
        data = pd.read_excel(file_stream, sheet_name='Sheet1', header=None, usecols=[0])

        # Extract numbers with prefixes
        scanned_numbers = data[0].map(extract_numbers_with_prefix).dropna().tolist()

        # Categorize numbers by prefix
        categorized_numbers = {}
        for num in scanned_numbers:
            match = re.match(r'(\D*)(\d+)', num)
            prefix = match.group(1) if match else "No Prefix"
            categorized_numbers.setdefault(prefix, []).append(num)

        results = {}
        total_missing = 0
        output_data = []

        for prefix, numbers in categorized_numbers.items():
            # Identify duplicates
            duplicates = sorted(set(x for x in numbers if numbers.count(x) > 1))

            # Extract numeric parts for missing number detection
            numeric_values = sorted(set(int(re.search(r'\d+', x).group()) for x in numbers if re.search(r'\d+', x)))

            # Determine range
            start_number, end_number = numeric_values[0], numeric_values[-1]

            # Generate full range
            total_range = set(range(start_number, end_number + 1))

            # Find missing numbers
            missing_numbers = sorted(total_range - set(numeric_values))
            total_missing += len(missing_numbers)

            # Store results per category
            results[prefix] = {
                "total_missing": len(missing_numbers),
                "missing_numbers": [f"{prefix}{mn}" for mn in missing_numbers],  # Add prefix to missing numbers
                "given_range": (start_number, end_number),
                "duplicates": duplicates
            }
            for mn in missing_numbers:
                output_data.append([prefix, start_number, end_number, f"{prefix}{mn}", "Missing"])
            for dn in duplicates:
                output_data.append([prefix, start_number, end_number, dn, "Duplicate"])

        results["Final Total"] = {"total_missing": total_missing}

        # Save results to Excel
        output_df = pd.DataFrame(output_data, columns=["Category", "Start Number", "End Number", "Number", "Status"])
        output_filename = "missing_numbers_report.xlsx"
        output_df.to_excel(output_filename, index=False)

        # Print results in structured format
        print(f"\n📄 Processing file: {uploaded_filename}")
        print("📁 Using first sheet: Sheet1")
        print("\n✅ File processed successfully!")
        for prefix, res in results.items():
            if prefix != "Final Total":
                print(f"\n📌 Category: {prefix if prefix else 'No Prefix'}")
                print(f"📌 Given range: {res['given_range'][0]} to {res['given_range'][1]}")
                print(f"🔢 Total missing numbers: {res['total_missing']}")
                print(f"📚 Missing book numbers: {', '.join(map(str, res['missing_numbers'])) if res['missing_numbers'] else 'None'}")
                if res["duplicates"]:
                    print(f"🔁 Duplicate numbers: {', '.join(map(str, res['duplicates']))}")
        print(f"\n📊 \033[1m Final Total Missing Numbers: {total_missing}")
        print("\n--------End of the report-------------")

        # Provide direct download
        print("\n📥 Downloading the report... ")
        files.download(output_filename)
        print("\n📥 Done. Check your Downloads... ")
        print("\n If any issues contact @ areddy.ragini@gmail.com\n \n =================END====================")


    except Exception as e:
        print(f"❌ Error processing file: {e}")

# Create a file upload widget
print("📂 Please upload an Excel file (.xlsx) to check for missing numbers in 'Sheet1'.\n")
upload_widget = widgets.FileUpload(
    accept='.xlsx',  # Only accept Excel files
    multiple=False   # Allow only a single file
)

def on_upload(change):
    if change['new']:
        process_file(change['new'])

# Display the file upload widget
display(upload_widget)
upload_widget.observe(on_upload, names='value')